In [ ]:
from google.colab import drive
drive.mount('/content/drive/')



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/treeleaf/DataScience

/content/drive/MyDrive/treeleaf/DataScience


In [ ]:
!pwd



/content/drive/MyDrive/treeleaf/DataScience


In [ ]:
import cv2
import numpy as np

: 

In [ ]:
def load_video_frames(video_path, img_size=(64, 64), max_frames=5000):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, img_size)
        frame = frame.astype('float32') / 255.0
        frames.append(frame)
        count += 1
    cap.release()
    return np.array(frames)

normal_frames = load_video_frames('normal_video.mp4')
anomaly_frames = load_video_frames('anomaly_video.mp4')

normal_frames = np.reshape(normal_frames, (len(normal_frames), 64, 64, 1))
anomaly_frames = np.reshape(anomaly_frames, (len(anomaly_frames), 64, 64, 1))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(normal_frames, test_size=0.2, random_state=42)

input_img = Input(shape=(64, 64, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(X_train, X_train, epochs=50, batch_size=128, shuffle=True, validation_split=0.2)


Epoch 1/50
2/2 [==============================] - 4s 1s/step - loss: 0.6914 - val_loss: 0.6827
Epoch 2/50
2/2 [==============================] - 3s 1s/step - loss: 0.6802 - val_loss: 0.6652
Epoch 3/50
2/2 [==============================] - 4s 1s/step - loss: 0.6606 - val_loss: 0.6335
Epoch 4/50
2/2 [==============================] - 2s 843ms/step - loss: 0.6254 - val_loss: 0.5791
Epoch 5/50
2/2 [==============================] - 2s 881ms/step - loss: 0.5658 - val_loss: 0.4927
Epoch 6/50
2/2 [==============================] - 2s 917ms/step - loss: 0.4733 - val_loss: 0.3720
Epoch 7/50
2/2 [==============================] - 2s 901ms/step - loss: 0.3492 - val_loss: 0.2388
Epoch 8/50
2/2 [==============================] - 3s 1s/step - loss: 0.2203 - val_loss: 0.1433
Epoch 9/50
2/2 [==============================] - 4s 1s/step - loss: 0.1373 - val_loss: 0.1187
Epoch 10/50
2/2 [==============================] - 2s 909ms/step - loss: 0.1215 - val_loss: 0.1309
Epoch 11/50
2/2 [=================

In [ ]:
def detect_anomalies(video_path, threshold):
    frames = load_video_frames(video_path)
    anomalies = []
    for i, frame in enumerate(frames):
        frame = np.expand_dims(frame, axis=-1)  # Add channel dimension
        frame = np.expand_dims(frame, axis=0)   # Add batch dimension
        reconstructed_frame = autoencoder.predict(frame)
        reconstructed_frame = np.squeeze(reconstructed_frame, axis=0)  # Remove batch dimension
        reconstructed_frame = np.squeeze(reconstructed_frame, axis=-1) # Remove channel dimension
        error = calculate_reconstruction_error(frame.squeeze(), reconstructed_frame)
        if error > threshold:
            anomalies.append((i, error))
    return anomalies

anomalies = detect_anomalies('anomaly_video.mp4', threshold)
print(f"Anomalies detected at frames: {anomalies}")


1/1 [==============================] - 0s 24ms/step
Anomalies detected at frames: [(0, 0.0034575693), (1, 0.003332428), (2, 0.0032629636), (3, 0.0033134688), (4, 0.0033553406), (5, 0.0034117727), (6, 0.0032952216), (7, 0.0032606698), (8, 0.0032786543), (9, 0.0032840434), (10, 0.0033206912), (11, 0.0032706019), (12, 0.0032381774), (13, 0.0031845216), (14, 0.0031951207), (15, 0.0032558672), (16, 0.0032312889), (17, 0.0032400896), (18, 0.0033212677), (19, 0.0033872079), (20, 0.0034721754), (21, 0.003386787), (22, 0.003316034), (23, 0.0033707093), (24, 0.0034057726), (25, 0.0034675978), (26, 0.0034075573), (27, 0.0033842924), (28, 0.0033968133), (29, 0.0034065363), (30, 0.0034433529), (31, 0.003375508), (32, 0.0033155303), (33, 0.0032332682), (34, 0.0032361497), (35, 0.0032867477), (36, 0.003263843), (37, 0.0032566756), (38, 0.0033184902), (39, 0.0033837976), (40, 0.0034658285), (41, 0.0033852058), (42, 0.0033142841), (43, 0.003368821), (44, 0.0034064096), (45, 0.003470234), (46, 0.0034092